In [1]:
import onnxruntime as ort 
from PIL import Image, ImageDraw
from torchvision.transforms import ToTensor
import torch
# print(onnx.helper.printable_graph(mm.graph))

im = Image.open('image_02.jpg').convert('RGB')
im = im.resize((640, 640))
im_data = ToTensor()(im)[None]
print(im_data.shape)
size = torch.tensor([[640, 640]])
sess = ort.InferenceSession("../model.onnx")

torch.Size([1, 3, 640, 640])


In [ ]:
output = sess.run(
    # output_names=['labels', 'boxes', 'scores'],
    output_names=None,
    input_feed={'images': im_data.data.numpy(), "orig_target_sizes": size.data.numpy()}
)

# print(type(output))
# print([out.shape for out in output])

labels, boxes, scores = output
print("labels shape = ",labels)
print("boxes shape = ",boxes)
print("scores shape = ",scores)
draw = ImageDraw.Draw(im)
thrh = 0.6

for i in range(im_data.shape[0]):

    scr = scores[i]
    lab = labels[i][scr > thrh]
    box = boxes[i][scr > thrh]

    print(i, sum(scr > thrh))

    for b in box:
        draw.rectangle(list(b), outline='red',)
        draw.text((b[0], b[1]), text=str(lab[i]), fill='blue', )

im.save('test.jpg')